The survey was fielded from May 8, 2023 to May 19, 2023.

Dataset is taken from the [Stack Overflow Annual Developer Survey](https://insights.stackoverflow.com/survey)

# Запитання 

1. Скільки респондентів пройшли опитування?
2. Скільки респондентів відповіли на всі запитання?
3. Які значення мір центральної тенденції для досвіду (WorkExp) респондентів?
4. Скільки респондентів працює віддалено?
5. Який відсоток респондентів програмує на Python?
6. Скільки респондентів навчалося програмувати за допомогою онлайн курсів?
7. Серед респондентів що програмують на Python в групуванні по країнам, яка середня та медіанна сума компенсації (ConvertedCompYearly) в кожній країні?
8. Які рівні освіти мають 5 респондентів з найбільшою компенсацією?

**Бонусні запитання**:  
1. В кожній віковій категорії, який відсоток респондентів програмує на Python?
2. Серед респондентів що знаходяться в 75 перцентилі за компенсацією середнього і працюють віддалено, які індустрії є найрозповсюдженішими?

In [2]:
import pandas as pd
import numpy as np
survey=pd.read_csv("C:/Users/User/Documents/data/Datasets/stack-overflow-developer-survey-2023/survey_results_public.csv")
structure=pd.read_csv("C:/Users/User/Documents/data/Datasets/stack-overflow-developer-survey-2023/survey_results_schema.csv")

In [3]:
#1. Скільки респондентів пройшли опитування?
u_resp=survey["ResponseId"].nunique()
print("Number of respondents by nunique() -", u_resp)
#The approach bellow shows number of raws (.shape returns list [Number of Raws, Number of Columns ]
# where Number of Raws is the 0-st value of the list and Number of Columns is the 1-st value of the list 
print("Number of respondents by .shape method -", survey.shape[0])

Number of respondents by nunique() - 89184
Number of respondents by .shape method - 89184


In [4]:
#2. Скільки респондентів відповіли на всі запитання?
survey_compl=survey.dropna()
print("Number of respondents answered all the questions is -", survey_compl["ResponseId"].count(), "\n (done without checking the schema)")

Number of respondents answered all the questions is - 0 
 (done without checking the schema)


In [5]:
#2. Скільки респондентів відповіли на всі запитання? 

#Test number of questions in 'survey' and 'structure'. See they are different - 78 vs 85
#print(structure['qname'].nunique())
#print(survey.shape[0]) #explanation of shape[] is in the first question above

#there should be a way to combine values for repeated question in 'survey' in order to mark both as non null if there is data in any of them

# Below is an approach, proposed by the course mentors, but I don't understand it
questions=set(structure.qname.unique()) & set(survey.columns) 
#probably, 'questions' is a set() object 
# and '&" check both sets - unique gnames from Structure and columns labels from Survey -
# and returns "True" if there is data in any of the columns with the same name

n=survey.dropna(subset=questions).shape[0]
#DO NOT understand "dropna(subset=questions)"
#Does it, while checking data by columns, consider two repeated columns labels as the same one?
#Thus, subset=questions specifies acual columns, some of which may by matrixes?

print("Number of respondents answered all the questions is -",n,"\n (with checking the schema)")

Number of respondents answered all the questions is - 2032 
 (with checking the schema)


In [6]:
#3. Які значення мір центральної тенденції для досвіду (WorkExp) респондентів?
print('Mean is',survey["WorkExp"].agg('mean').round(2))
print('Median is', survey["WorkExp"].median())
#print('Mode is', survey["WorkExp"].mode())

Mean is 11.41
Median is 9.0


In [17]:
#4. Скільки респондентів працює віддалено?
remote=survey.groupby('RemoteWork') #group all the raws in 3 groups (by values from 'RemoteWork' column)
r_count=remote['ResponseId'].count() #counts number of answers for each of 3 variants by column 'ResponseId'

#remote=survey.groupby('RemoteWork')['ResponseId'].count()
#print(remote,'\n')

print('Work remotely',r_count['Remote']) #prints only one of 3 variants - 'Remote'


RemoteWork
Hybrid (some remote, some in-person)    31131
In-person                               12113
Remote                                  30566
Name: ResponseId, dtype: int64 

Work remotely 30566


In [8]:
#5. Який відсоток респондентів програмує на Python?
#subset=survey.loc[:,['ResponseId','LanguageHaveWorkedWith']]
l_py=survey['LanguageHaveWorkedWith'].str.contains('Python')
l_py1=l_py[l_py == True]

py_perc_gen=l_py.sum()/u_resp*100

print("Calculation 1 with .count() method:",round(l_py1.count()/u_resp*100,2),'%')
print("Calculation 2 with .sum() method:",round(l_py.sum()/u_resp*100,2),'%')
print("Calculation 3 with additional variable:",round(py_perc_gen,2),'%')


Calculation 1 with .count() method: 48.39 %
Calculation 2 with .sum() method: 48.39 %
Calculation 3 with additional variable: 48.39 %


In [9]:
#6. Скільки респондентів навчалося програмувати за допомогою онлайн курсів?
online_courses=survey['LearnCode'].str.contains('Online Courses').sum() 
# gets the list of True/False and then the function "sum()" summs Trues (True=1)
print("The number of interviewees who learnt coding with online courses is",online_courses)

The number of interviewees who learnt coding with online courses is 43201


In [10]:
#6. Скільки респондентів навчалося програмувати за допомогою онлайн курсів?
online_or_not=survey['LearnCode'].str.contains('Online Courses') #get the list of True/False
online_courses=online_or_not[online_or_not==True].count()
print("Another way to calculate online courses repeats the previous result -",online_courses)

Another way to calculate online courses repeats the previous result - 43201


In [11]:
#7. Серед респондентів що програмують на Python в групуванні по країнам, 
#яка середня та медіанна сума компенсації (ConvertedCompYearly) в кожній країні?

# Create True/False series 
l_py=survey['LanguageHaveWorkedWith'].str.contains('Python') 

#Add column with Python "True\False" to the DataFrame
survey['Python']=l_py 

#Select rows in DataFrame with Python True
subset_py=survey[survey['Python']==True] 

#Group them by countries
c_group=subset_py.groupby('Country') 

#Calculations with compensation
c_comp_py=c_group['ConvertedCompYearly'].agg(['mean','median']) 
print("The table shows mean and median compensation for Python partitioners by country: \n",c_comp_py)

The table shows mean and median compensation for Python partitioners by country: 
                                               mean   median
Country                                                    
Afghanistan                             665.000000     48.0
Albania                               28008.600000  11844.0
Algeria                                8336.333333   6586.0
Andorra                               32127.000000  32127.0
Angola                                  662.000000    662.0
...                                            ...      ...
Venezuela, Bolivarian Republic of...  24973.529412  12000.0
Viet Nam                              20191.870370  13401.0
Yemen                                  8373.000000   9000.0
Zambia                                13051.000000   9687.0
Zimbabwe                               5600.000000   6000.0

[173 rows x 2 columns]


In [12]:
#8. Які рівні освіти мають 5 респондентів з найбільшою компенсацією?

#Sort values in boundaries of the entire DataFrame
comp_sorted=survey.sort_values(by='ConvertedCompYearly', ascending=False)[:5]
comp_sorted.loc[:,['ConvertedCompYearly','EdLevel']]

print("Ed level of the 5 most high compensated respondents is: \n",comp_sorted.loc[:,['ConvertedCompYearly','EdLevel']])

Ed level of the 5 most high compensated respondents is: 
        ConvertedCompYearly                                         EdLevel
53268           74351432.0  Professional degree (JD, MD, Ph.D, Ed.D, etc.)
77848           73607918.0  Professional degree (JD, MD, Ph.D, Ed.D, etc.)
66223           72714292.0    Bachelor’s degree (B.A., B.S., B.Eng., etc.)
28121           57513831.0                       Primary/elementary school
19679           36573181.0  Professional degree (JD, MD, Ph.D, Ed.D, etc.)


In [13]:
#8. Які рівні освіти мають 5 респондентів з найбільшою компенсацією?

comp_sorted=survey.sort_values(by='ConvertedCompYearly', ascending=False).iloc[0:5,7]

print("Another approach shows the same result:\n",comp_sorted)

Another approach shows the same result:
 53268    Professional degree (JD, MD, Ph.D, Ed.D, etc.)
77848    Professional degree (JD, MD, Ph.D, Ed.D, etc.)
66223      Bachelor’s degree (B.A., B.S., B.Eng., etc.)
28121                         Primary/elementary school
19679    Professional degree (JD, MD, Ph.D, Ed.D, etc.)
Name: EdLevel, dtype: object


## Bonus ##

In [14]:
#1. В кожній віковій категорії, який відсоток респондентів програмує на Python?
age_cat=survey.groupby('Age')
py_in_age=age_cat['Python'].sum()
py_perc=py_in_age/age_cat['Python'].count()*100
#rounded_py_perc=round(py_perc,2)
#print(py_in_age)
print('Percent of Python usage by',py_perc)
#Connot round


Percent of Python usage by Age
18-24 years old       63.054187
25-34 years old       48.361257
35-44 years old       42.230384
45-54 years old       39.363793
55-64 years old       37.697929
65 years or older     32.879201
Prefer not to say     57.632399
Under 18 years old    71.234599
Name: Python, dtype: object


In [15]:
#1. В кожній віковій категорії, який відсоток респондентів програмує на Python?
age_cat=survey.groupby('Age')
py_in_age=age_cat['Python'].sum()
py_perc=py_in_age/age_cat['ResponseId'].count()*100
#rounded_py_perc=round(py_perc,2)
#print(py_in_age)
print('Percent of Python usage by',py_perc)

Percent of Python usage by Age
18-24 years old       61.390887
25-34 years old       47.577225
35-44 years old       41.442626
45-54 years old       38.456923
55-64 years old       36.497642
65 years or older     30.913749
Prefer not to say     41.202673
Under 18 years old    68.628876
dtype: object


In [16]:
#2.Серед респондентів що знаходяться в 75 перцентилі за компенсацією середнього і працюють віддалено, 
#які індустрії є найрозповсюдженішими?

#*first_quartile_age = students_df['Вік'].quantile(0.25)  # 21.0* Знаходить перший квартиль віку студентів.

sort_comp=survey.sort_values(by='ConvertedCompYearly', inplace=False)

#Test - find number (value) that could be equal the 3rd quartile in Compensation column
third_q_comp=sort_comp['ConvertedCompYearly'].quantile(0.75)
#print('1. number (value) that could be equal the 3rd quartile in Compensation column',third_quartile_comp)

#Test - check if there is this value in the dataset
#print(survey[survey['ConvertedCompYearly']==third_q_comp])

#Test - check if the value (3rd quartile that was calculated above) exists in a the column ConvertedCompYearly - series in which will look for
#print('\n 2. if the 3rd quartile value exists in a th column ConvertedCompYearly  \n',third_q_comp in sort_comp['ConvertedCompYearly'],' \n')

#Limit DataFrame to rows with ConvertedCompYearly above quantile(0.75)
subset_third_q=sort_comp[sort_comp['ConvertedCompYearly']>third_q_comp]
#Limit DataFrame 'subset_third_q' to rows with Remote in RemoteWork
subset_third_q_r= subset_third_q[subset_third_q['RemoteWork']=='Remote']
#Create subset with grouped limited above data 
Subset_ind_groups=subset_third_q_r.groupby('Industry') # if we add .count() it returns the entire data set, thus we should create series

Ind_counts=Subset_ind_groups['ResponseId'].count()
print('Industries by popularity among respondents of the percentile 75 who work remotely:  \n',Ind_counts.sort_values(ascending=False),' \n')

Industries by popularity among respondents of the percentile 75 who work remotely:  
 Industry
Information Services, IT, Software Development, or other Technology    1800
Financial Services                                                      535
Other                                                                   441
Healthcare                                                              365
Retail and Consumer Services                                            243
Manufacturing, Transportation, or Supply Chain                          168
Advertising Services                                                    121
Insurance                                                                96
Higher Education                                                         65
Legal Services                                                           21
Oil & Gas                                                                17
Wholesale                                                            